In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import seaborn as sns
from ionpy.analysis import ResultsLoader
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [ ]:
from ese.experiment.analysis.inference import load_cal_inference_stats

path = f"{root}/11_03_23_WMH_Inference"

pix_inference_dicts, image_inference_dfs, metadata_df = load_cal_inference_stats(
    path,
    return_metadata=True
)

In [ ]:
metadata_df['log_set']

In [ ]:
# Get the pixel preds out
# pixel_preds_dict = pix_inference_dicts[metadata_df['log_set'][0]]
image_pred_df = image_inference_dfs[image_inference_dfs['log_set'] == metadata_df['log_set'][0]]

## Pixel-level Analysis

In [ ]:
# from ese.experiment.analysis.err_diagrams import viz_accuracy_vs_confidence

# viz_accuracy_vs_confidence(
#     pixel_preds_dict,
#     title="WMH Confidence vs Accuracy per (Bin and Predicted Label)",
#     x="pred_label",
#     hue="measure",
#     col="bin_num",
#     kind="bar",
#     add_average=True,
#     add_proportion=True,
#     facet_kws={'sharey': False, 'sharex': False}
#     )

In [ ]:
# from ese.experiment.analysis.err_diagrams import viz_accuracy_vs_confidence

# viz_accuracy_vs_confidence(
#     pixel_preds_dict,
#     title="WMH Confidence vs Accuracy per (Bin and Num Neighbors)",
#     x="num_neighbors",
#     hue="measure",
#     col="bin_num",
#     kind="bar",
#     add_average=True,
#     add_proportion=True,
#     facet_kws={'sharey': False, 'sharex': False},
#     )

## Image-level Analysis

In [ ]:
image_pred_df.head()

In [ ]:
from ese.experiment.analysis.err_diagrams import viz_cal_metric_corr

viz_cal_metric_corr(
    image_pred_df,
    title="WMH Calibration Metric Correlation",
    )